# Install packages 

In [6]:
requirements = [
    "folium",
    "rioxarray",
    "earthpy",
    "pandas",
    "turfpy"
]

In [7]:
with open('requirements.txt', 'w') as req_file:
    for req in requirements:
      req_file.write(req + "\n")

In [8]:
!ls

requirements.txt  sample_data


In [9]:
!pip install -r 'requirements.txt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import - Global Vars

In [38]:
import pandas as pd
import os
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es
from branca.element import Figure
from folium.plugins import FastMarkerCluster
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, MultiPolygon, Feature
import urllib.request, json
import geojson
from datetime import datetime

In [18]:
#MTL central(?) lat-long
mtl_lat_long = { "Latitude": 45.508888, "Longitude": -73.561668 }

#Datasets from: https://donnees.montreal.ca/ville-de-montreal/interventions-service-securite-incendie-montreal
sim_2016_2022_url = "https://data.montreal.ca/dataset/2fc8a2b9-1556-410e-a118-c46e97e9f19e/resource/71e86320-e35c-4b4c-878a-e52124294355/download/donneesouvertes-interventions-sim.csv"
sim_2005_2014_url = "https://data.montreal.ca/dataset/2fc8a2b9-1556-410e-a118-c46e97e9f19e/resource/0a778ac0-9b5a-42cb-8557-167f7f9b8feb/download/donneesouvertes-interventions-sim-2005-2014.csv"
ff_stations_url = "https://data.montreal.ca/dataset/c69e78c6-e454-4bd9-9778-e4b0eaf8105b/resource/5b9c0e1d-3f75-4e98-b53d-6e979c18cc98/download/casernes.csv"
mtl_arr_url = "https://data.montreal.ca/dataset/9797a946-9da8-41ec-8815-f6b276dec7e9/resource/e18bfd07-edc8-4ce8-8a5a-3b617662a794/download/limites-administratives-agglomeration.geojson"


# Clean Data

## Incidents SIM data MTL

In [36]:
incidents_new = pd.read_csv(sim_2016_2022_url, parse_dates=['CREATION_DATE_TIME'])

In [43]:
#merge (foreign key / casernes dataset)
incidents_new['BOROUGH'] = pd.DataFrame(incidents_new["NOM_ARROND"].mask(incidents_new["NOM_ARROND"].str.match("Indéterminé"),incidents_new["NOM_VILLE"]))
incidents_new["BOROUGH"] = incidents_new["BOROUGH"].str.upper()

total_inc = incidents_new.groupby('BOROUGH')['INCIDENT_NBR'].count().to_dict()
total_last_yr = incidents_new[incidents_new['CREATION_DATE_TIME'].dt.year == datetime.now().year - 1].groupby('BOROUGH')['INCIDENT_NBR'].count().to_dict()

#save -> report
#incidents_new.to_csv("incidents_new_merged.csv")

## Firefighters Stations MTL

In [49]:
ff_stations_mtl = pd.read_csv(ff_stations_url)

ff_stations_mtl = ff_stations_mtl.fillna(value={"ARRONDISSEMENT":"Indéterminé","VILLE":"Indéterminé"})
ff_stations_mtl['BOROUGH'] = pd.DataFrame(
    ff_stations_mtl["ARRONDISSEMENT"].mask(ff_stations_mtl["ARRONDISSEMENT"].str.match("Indéterminé"),
                                            ff_stations_mtl["VILLE"]))
ff_stations_mtl["BOROUGH"] = ff_stations_mtl["BOROUGH"].str.upper()

## Get Geo Data MTL

In [20]:
web_url = urllib.request.urlopen(mtl_arr_url)
data = web_url.read()

encoding = web_url.info().get_content_charset('utf-8')
mtl_geo = json.loads(data.decode(encoding))

# Plot Data

In [51]:
m_arro = folium.Map(location=[mtl_lat_long["Latitude"], mtl_lat_long["Longitude"]],zoom_start=11, tiles="Stamen Toner")

folium.GeoJson(
    mtl_arr_url,
    tooltip=folium.GeoJsonTooltip(fields=['NOM_OFFICIEL'], aliases=['Name: ']),
    style_function=lambda x: {'fillColor':'#0000ff'},
    highlight_function=lambda x: {'fillColor':'red'},
).add_to(m_arro)

for index, row in ff_stations_mtl.iterrows():
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        tooltip=f"<b>Caserne No. :</b>{row['CASERNE']}<br>"
                f"<b>Incidents Last Yr :</b>: {total_last_yr[row['BOROUGH']]}<br>"
                f"<b>Total Incidents :</b>: {total_inc[row['BOROUGH']]}"
    ).add_to(m_arro)

m_arro